In [154]:
from typing import Dict, List
import pandas as pd
import datetime as dt
import os
import json
from dotenv import load_dotenv
import pandas as pd

from specklepy.api.client import SpeckleClient
from specklepy.api.credentials import get_default_account
from specklepy.objects import Base
from specklepy.objects.geometry import Point, Line
from specklepy.transports.server import ServerTransport
from specklepy.api import operations
import itertools

In [155]:

speckle_server = "app.speckle.systems"
speckle_token = os.environ["TOKEN"]
client = SpeckleClient(host=speckle_server)
client.authenticate_with_token(speckle_token)
stream_id = "013aaf06e7"
transport = ServerTransport(client=client, stream_id=stream_id)

In [156]:

object_id = "eb80959eb9eebaafb7a4f1db38e841d1"
obj = client.object.get(stream_id, object_id)

In [157]:
from specklepy.serialization.base_object_serializer import BaseObjectSerializer
serializer = BaseObjectSerializer()

In [158]:
hash_id, obj_data = serializer.traverse_base(obj)
obj_data["userStrings"] 

{'id': 'f3758c29e2daf3477efd27e414e5fb8d',
 'speckle_type': 'Base',
 'totalChildrenCount': 0,
 'Bottom z (mPD)': -50,
 'Domain Margin (m)': 20,
 'Max Easting (m)': 838845,
 'Max Northing (m)': 821245,
 'Min Easting (m)': 837755,
 'Min Northing (m)': 819780,
 'Top z (mPD)': 0,
 'Voxel Size (m)': 5,
 'applicationId': None,
 'units': None}

In [159]:

x_min = obj_data["userStrings"]['Min Easting (m)']
x_max = obj_data["userStrings"]['Max Easting (m)']
y_min = obj_data["userStrings"]['Min Northing (m)']
y_max = obj_data["userStrings"]['Max Northing (m)']
z_min = obj_data["userStrings"]['Bottom z (mPD)']
z_max = obj_data["userStrings"]['Top z (mPD)']
voxel_size = obj_data["userStrings"]["Voxel Size (m)"]

x_points = int((x_max - x_min) / voxel_size)
y_points = int((y_max - y_min) / voxel_size)
z_points = int((z_max - z_min) / voxel_size)

x_points, y_points, z_points


(218, 293, 10)

In [175]:
x_arr = [x_min + i * voxel_size for i in range(x_points)]
y_arr = [y_min + i * voxel_size for i in range(y_points)]
z_arr = [z_min + i * voxel_size for i in range(z_points)]

df = pd.DataFrame(list(itertools.product(x_arr, y_arr, z_arr)), columns=["x", "y", "z"])
df = df + 2.5
df.to_csv("points.csv", index=False)


    
    
    
    


In [174]:
df

,x,y,z
0,837757.5,819782.5,-47.5
1,837757.5,819782.5,-42.5
2,837757.5,819782.5,-37.5
3,837757.5,819782.5,-32.5
4,837757.5,819782.5,-27.5
...,...,...,...
638735,838842.5,821242.5,-22.5
638736,838842.5,821242.5,-17.5
638737,838842.5,821242.5,-12.5
638738,838842.5,821242.5,-7.5


In [170]:
obj = Base()
obj["voxels"] = points

hash_id = operations.send(obj, [transport])


Upload server response: Error inserting object in the database: Object too large. (139.117703 MB is > than limit, 10 MB)
Error sending batch of objects to server: SpeckleException: Could not save the object to the server - status code 400 (Error inserting object in the database: Object too large. (139.117703 MB is > than limit, 10 MB))


SpeckleException: SpeckleException: Could not save the object to the server - status code 400 (Error inserting object in the database: Object too large. (139.117703 MB is > than limit, 10 MB))

In [161]:
object_id = "1a98b24773c42b01be65b71f63ee14bc"
start_end_points = operations.receive(object_id, transport)

In [162]:
_, start_end_points_data = serializer.traverse_base(start_end_points)
with open("start_end_points.json", "w") as f:
    json.dump(start_end_points_data, f)

In [163]:
start_point = start_end_points_data["elements"][1]["elements"][0]
end_points = start_end_points_data["elements"][0]

start = {
    "x" : start_point["x"],
    "y" : start_point["y"],
    "z" : start_point["z"]
}
end_points = [
    {
        "x" : end_point["x"],
        "y" : end_point["y"],
        "z" : end_point["z"]
    }
    for end_point in end_points["elements"]
]

In [164]:
end_points

[{'x': 838545.0, 'y': 821225.0, 'z': 0.0},
 {'x': 837775.0, 'y': 820755.0, 'z': 0.0}]

In [165]:
start_index = {
    "x" : int((start["x"] - x_min) / voxel_size),
    "y" : int((start["y"] - y_min) / voxel_size),
    "z" : int((start["z"] - z_min) / voxel_size)
}

In [166]:
start_index

{'x': 214, 'y': 4, 'z': 10}

In [167]:
end_indices = [
    {
        "x" : int((end_point["x"] - x_min) / voxel_size),
        "y" : int((end_point["y"] - y_min) / voxel_size),
        "z" : int((end_point["z"] - z_min) / voxel_size)
    }
    for end_point in end_points
]



In [168]:
# In summary the data is as follows:



